In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd
from requests.auth import HTTPProxyAuth
import time

In [2]:
def getSoup(page):
    print("URL: ",page)


    heads = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'}
    time.sleep(1000)
    response = requests.get(page,headers=heads)
    print("request Stats: ",response.status_code )
    if(response.status_code == 403):
        print("not response: ",response.status_code)
        exit(0)
    soup= BeautifulSoup(response.content, 'html.parser')
    return soup
    pass


In [3]:
def getft(page):
    soup=getSoup(page)
    print(soup)
    ft = soup.select_one('.hauptlink > .inline-table > tr > td:nth-child(2) > a')['href']
    # ft = soup.select_one('#yw1 > table > tbody > tr:nth-child(2) > td:nth-child(1) > table > tbody > tr > td:nth-child(2) > a')['href']
    ft="https://www.transfermarkt.de"+ft
    
    return ft
    pass


In [4]:
def getClubs(page):
    soup=getSoup(page)
    data=[]
    links=[]
    clubs = soup.select('#yw1 .no-border-links a:nth-child(1)')
    for club in clubs:
        links.append('https://www.transfermarkt.de'+club['href'])
    data.append(links)
    prem_table_link=soup.select_one('.tab-print .table-footer a')['href']
    data.append("https://www.transfermarkt.de"+prem_table_link)
    wanted_page=soup.select_one("#main > main > header > div.data-header__box--big > div > span.data-header__club > a")["href"]
    data.append(wanted_page)
    return data

In [5]:
def getPlayers(page):
    soup=getSoup(page)
    links=[]
    players = soup.select('.hauptlink a')
    for player in players:
        links.append('https://www.transfermarkt.de'+player['href'])
    return links

In [6]:
def getClubTable(page,wanted_page_link):
    soup=getSoup(page)
    # https://www.transfermarkt.de/manchester-city/startseite/verein/281/saison_id/2020
    data=[]
    club_id=re.sub(r"https.*verein\/","",page)
    club_id=re.sub(r"\/.*","",club_id)
    name =  soup.select_one('#main > main > header > div.data-header__headline-container > h1').text.strip()
    country_id=re.sub(r".*\/","n-",wanted_page_link)
    #country_id                     #main > main > div.row.vereinsstartseite > div.large-4.columns > div.box.daten-und-fakten-verein.viewport-tracking > div > span:nth-child(14)
                                   #main > main > div.row.vereinsstartseite > div.large-4.columns > div.box.daten-und-fakten-verein.viewport-tracking > div > span:nth-child(14)
    try:
        if soup.select_one("#main > main > div.row.vereinsstartseite > div.large-4.columns > div.box.daten-und-fakten-verein.viewport-tracking > div > span:nth-child(16)")!=None:
            fd=soup.select_one("#main > main > div.row.vereinsstartseite > div.large-4.columns > div.box.daten-und-fakten-verein.viewport-tracking > div > span:nth-child(16)").text
        elif soup.select_one("#main > main > div.row.vereinsstartseite > div.large-4.columns > div.box.daten-und-fakten-verein.viewport-tracking > div > span:nth-child(14)")!=None:
            fd=soup.select_one("#main > main > div.row.vereinsstartseite > div.large-4.columns > div.box.daten-und-fakten-verein.viewport-tracking > div > span:nth-child(14)").text
        else: fd=soup.select_one("#main > main > div.row.vereinsstartseite > div.large-4.columns > div.box.daten-und-fakten-verein.viewport-tracking > div > span:nth-child(12)").text
    except:
        fd=None
    try:
        value = soup.select_one('.data-header__market-value-wrapper ').text.strip().replace("€","")
        value=re.sub(r"\s.*","",value)
        if "k"in value:
            n_value=value.replace("k","")
            float(n_value)*1000
        if "m" in value:
            n_value=value.replace("m","")
            float(n_value)*1000000
        if "bn" in value:
            n_value=value.replace("bn","")
            float(n_value)*1000000000
    except:
        n_value=None
        value=None
    stadium = soup.select_one("#main > main > header > div.data-header__info-box > div > ul:nth-child(2) > li:nth-child(2) > span > a").text
    squad_size=soup.select_one("#main > main > header > div.data-header__info-box > div > ul:nth-child(1) > li:nth-child(1) > span").text.strip()
    average_age=soup.select_one("#main > main > header > div.data-header__info-box > div > ul:nth-child(1) > li:nth-child(2) > span").text.strip()
    national_players=soup.select_one("#main > main > header > div.data-header__info-box > div > ul:nth-child(2) > li:nth-child(1) > span > a").text
    ##                                        main > main > header > div.data-header__info-box > div > ul:nth-child(2) > li:nth-child(3) > span > span > a
    try:                                         #main > main > header > div.data-header__info-box > div > ul:nth-child(2) > li:nth-child(3) > span > span > a
        current_transfer_record=soup.select_one("#main > main > header > div.data-header__info-box > div > ul:nth-child(2) > li:nth-child(3) > span > span > a").text.replace("€","")
        # print(current_transfer_record,"*****",club_id)
        if "k" in current_transfer_record:
           cnt=current_transfer_record.replace("k","")
           cnt=float(cnt)*1000
        if "m" in current_transfer_record:
           cnt=current_transfer_record.replace("m","")
           cnt=float(cnt)*1000000
        if "bn" in current_transfer_record:
           cnt=current_transfer_record.replace("bn","")
           cnt=float(cnt)*1000000000
        if current_transfer_record=="+-0":
            cnt=float(0)
    except:
        cnt=None
    try:
        foriegners = int(soup.select_one("#main > main > header > div.data-header__info-box > div > ul:nth-child(1) > li:nth-child(3) > span > a").text)
    except:
        foriegners=None
    #country = soup.select_one('#main > main > div.row.vereinsstartseite > div.large-4.columns > div.box.daten-und-fakten-verein.viewport-tracking > div > div:nth-child(8) > span:nth-child(2)').text
    # manager = soup.select_one(".container-main >a").text
    try:
        n_value=re.sub(r"\s.*","",value)
    except:
        n_value=None
    data.append("c-"+str(club_id))
    data.append(name)
    data.append(str(country_id))
    data.append(fd)
    data.append(n_value)
    data.append(str(stadium))
    data.append(int(squad_size))
    try:
        data.append(float(average_age))
    except:
        data.append(average_age)
    data.append(int(national_players))
    data.append(cnt)
    data.append(foriegners)
    return data
    #return name,foundation_date,value,stadium,squad_size,average_age,national_players,current_transfer_record,foriegners

In [7]:
def getClubSeason(page):
    soup=getSoup(page)
    data=[]
    table=soup.select("#yw1 > table > tbody > tr ")
    for row in table:
        row_data = []
        cells = row.find_all('td')
        for i in range(10):
            if i==2:
                cells[i]=cells[i].select('a')[0]['href']
                row_data.append(cells[i])
            else:
                # print(cells[i])
                row_data.append(cells[i].text.strip())
        data.append(row_data)
    # print(data,"#######\n")
    final=[]
    for d in data:
        
        #id
        temp=re.sub(r"^\/.*verein\/","",d[2]).split("/saison_id/")
        club_id = "c-"+temp[0]
        season_id = "s-"+temp[1]
        temp2=re.sub(r"https.*\/wettbewerb\/","",page)
        competition_id=re.sub(r"\/.*","",temp2)
        rank =d[0]
        matches = d[3]
        win = d[4]
        draw = d[5]
        loss=d[6]
        goals_scored=d[7].split(':')[0]
        goals_conceded=d[7].split(':')[1]
        # players_avg_age = soup.select_one("#main > main > header > div.data-header__info-box > div > ul:nth-child(1) > li:nth-child(3) > span > a").text
        players_avg_age = soup.select_one("#main > main > header > div.data-header__info-box > div > ul:nth-child(2) > li:nth-child(2) > span").text.split()[0]
        final.append([str(club_id),str(season_id),str(competition_id),int(rank),int(matches),int(win),int(draw),int(loss),int(goals_scored),int(goals_conceded),float(players_avg_age)])
    return final

In [8]:
def getTransferTable(page):
    soup = getSoup(page)
    rows = soup.select(".viewport-tracking:nth-child(8) > div") 
    rows = rows[1:-1]
    data = []
    for row in rows:
        player_id="p-"+re.sub(r".*\/","",page)
        row_data = []
        row_data.append(player_id)
        cells = row.select("div")  
        
        for i in range(6):
            if i==0:
                row_data.append("s-20"+str(re.sub(r"\/.*","",cells[i].text.strip())))
            if i==1:
                continue
            elif i == 2 or i == 3:
                a_tag = cells[i].find("a")  
                href = a_tag.get("href") if a_tag else ""  
                href=re.sub(r".*verein\/","",href)
                href=re.sub(r"\/.*","",href)
                row_data.append("c-"+str(href))
            elif i==4 or i==5:
                d=cells[i].text.strip().replace("€","")
                if "Loan fee:" in d:
                    d=d.replace("Loan fee:\n","")
                if "m" in d:
                    d=float(d.replace("m",""))*1000000
                elif "k" in d:
                    d=float(d.replace("k",""))*1000
                elif d=="-" or d=="End of loan" or d=="?":
                    d=None
                elif d=="free transfer":
                    d=0
                else:
                    d=None
                row_data.append(d)  
        
        data.append(row_data)  
    data=data[1:-1]
    return data

In [9]:
def main():
    df1 = pd.DataFrame(columns = ['club_id', 'name', 'country_id','foundation_date','value','stadium','squad_size','average_age','national_players','current_transform_record','foriegners'])
    df2 = pd.DataFrame(columns = ['club_id', 'season_id', 'competition_id','rank','matches','win','draw','loss','goals_scored','goals_conceded','players_avg_age'])
    df3 = pd.DataFrame(columns=['player_id','season_id','origin_club_id','destination_club_id','market_value','fee'])
    df4 = pd.DataFrame()
    # clubs_links=[]
    countries_links=["https://www.transfermarkt.de/wettbewerbe/national/wettbewerbe/189",
                     "https://www.transfermarkt.de/wettbewerbe/national/wettbewerbe/40",
                     "https://www.transfermarkt.de/wettbewerbe/national/wettbewerbe/75",
                     "https://www.transfermarkt.de/wettbewerbe/national/wettbewerbe/50",
                     "https://www.transfermarkt.de/wettbewerbe/national/wettbewerbe/157"]
    # countries_links=["https://www.transfermarkt.de/wettbewerbe/national/wettbewerbe/136",
    #                   "https://www.transfermarkt.de/wettbewerbe/national/wettbewerbe/174",
    #                   "https://www.transfermarkt.de/wettbewerbe/national/wettbewerbe/122"]    
    # countries_links=["https://www.transfermarkt.de/wettbewerbe/national/wettbewerbe/189"]

    seasons=[2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
    # seasons=[2015]
    for i in countries_links:
        for season in seasons:
            print(i,season)

            country_detailed=i+f"/saison_id/{season}/plus/1"
            ft_link=getft(country_detailed)
            getClubsdata=getClubs(ft_link)
            prem_table_link=getClubsdata[1]
            clubs=getClubsdata[0]
            wanted_page_link=getClubsdata[2]

            for club in clubs:
                print(club)
                club_table_data=getClubTable(club,wanted_page_link)
                df1.loc[len(df1)] = club_table_data
                players=getPlayers(club)
                df4.loc[len(df4)]=players
                
                for player in players:

                    print(club,player)
                    transfer_list=getTransferTable(player)
                    for transfer in transfer_list:
                        print(transfer)
                        df3.loc[len(df3)]=transfer
                         
            
            club_season_data=getClubSeason(prem_table_link)
            for list in club_season_data:
                df2.loc[len(df2)] = list
    df1.to_csv('club.csv')
    df2.to_csv('season_club.csv')
    print("finished")
    df3.to_csv('transfer.csv')
    
    print(df1,"\n###########################################",df2,'\n%%%%%%%%%%%%%%%%%%%%%%%%%%')
    pass

In [ ]:
if __name__ == "__main__":
    main()